FUNCIONES UTILS

In [ ]:
from models.diagnosis import Diagnosis
from models.patient import Patient
from models.eye_examination import EyeExamination
from models.eye_image import EyeImage
from models.retina import Retina
from models.contours import Contours
import pandas as pd
import os

# Convierte el excel de diagnosis_data.xlsx en un DataFrame de pandas
def convert_xls_to_df(path_file: pd.ExcelFile, sheet: str) -> pd.DataFrame:
    df = pd.read_excel(path_file, sheet_name=sheet)
    df_optimazing = optimazing_df(df)
    print(df_optimazing)
    return df_optimazing

# Realiza la limpieza del código ID a formato int y aplica tipo de dato a las columnas del DataFrame
def optimazing_df(df: pd.DataFrame) -> pd.DataFrame:
    df['ID'] = df['ID'].str.replace('#', '').astype('int')
    df['Age'] = df['Age'].astype('Int8')
    df['Gender'] = df['Gender'].astype('category')
    df['Diagnosis'] = df['Diagnosis'].astype('category')
    df['dioptre_1'] = df['dioptre_1'].astype('Float32')
    df['dioptre_2'] = df['dioptre_2'].astype('Float32')
    df['astigmatism'] = df['astigmatism'].astype('Float32')
    df['Phakic/Pseudophakic'] = df['Phakic/Pseudophakic'].astype('category')
    df['Pneumatic'] = df['Pneumatic'].astype('Float32')
    df['Perkins'] = df['Perkins'].astype('Float32')
    df['Pachymetry'] = df['Pachymetry'].astype('Float32')
    df['Axial_Length'] = df['Axial_Length'].astype('Float32')
    df['VF_MD'] = df['VF_MD'].astype('Float32')
    return df

# Carga los datos de patient, diagnosis, EyeExamination (OD), EyeImage (OD), Contours (OD), Retina (OD). De df_entry_od a la lista diagnosises[]
def load_data(df_entry_od: str) -> list:
    try:
        diagnosises = []
        for index, row in df_entry_od.iterrows():
            diagnosis = Diagnosis(
                diagnosis_id=row['ID'],
                diagnosis=row['Diagnosis'],
            )
            patient = Patient(
                age=row['Age'],
                gender=row['Gender']
            )
            diagnosis.add_patient(patient=patient)
            eye_examination = EyeExamination(
                dioptre_1=row['dioptre_1'],
                dioptre_2=row['dioptre_2'],
                astigmatism=row['astigmatism'],
                phakic_pseudophakic=row['Phakic/Pseudophakic'],
                pneumatic=row['Pneumatic'],
                perkins=row['Perkins'],
                pachymetry=row['Pachymetry'],
                axial_length=row['Axial_Length'],
                vf_md=row['VF_MD']
            )
            diagnosis.add_eye_examination_od(eye_examination_od=eye_examination)
            eye_image = EyeImage(
                path_retina_contours_image=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'ImagesWithContours', f'Opht_cont_RET{row['ID']:03d}OD.jpg'),            
            )
            retina_image = Retina(
                path_retina_image=os.path.join('..', 'data', 'load_data', 'FundusImages', f'RET{row['ID']:03d}OD.jpg')
            )
            contours_layers = Contours(
                path_retina_cup_exp1=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row['ID']:03d}OD_cup_exp1.txt'),
                path_retina_cup_exp2=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row['ID']:03d}OD_cup_exp2.txt'),
                path_retina_disc_exp1=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row['ID']:03d}OD_disc_exp1.txt'),
                path_retina_disc_exp2=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row['ID']:03d}OD_disc_exp2.txt')
            )
            eye_examination.add_eye_image(eye_image=eye_image)
            eye_image.add_retina_image(retina_image=retina_image)
            eye_image.add_contours_layer(contours_layer=contours_layers)
            diagnosises.append(diagnosis)
    except Exception as e:
        print(e)
    else: 
        return diagnosises

# Agrega EyeExamination (OS), EyeImage (OS), Contours (OS), Retina (OS). De df_entry_os a la lista diagnosises[]
def add_diagnosises_os(diagnosises: list[Diagnosis], df_entry_os: pd.DataFrame):
    for index, row_os in df_entry_os.iterrows():
        for diagnosis in diagnosises:
            if row_os['ID']==diagnosis.diagnosis_id:
                eye_examination = EyeExamination(
                    dioptre_1=row_os['dioptre_1'],
                    dioptre_2=row_os['dioptre_2'],
                    astigmatism=row_os['astigmatism'],
                    phakic_pseudophakic=row_os['Phakic/Pseudophakic'],
                    pneumatic=row_os['Pneumatic'],
                    perkins=row_os['Perkins'],
                    pachymetry=row_os['Pachymetry'],
                    axial_length=row_os['Axial_Length'],
                    vf_md=row_os['VF_MD']
                )
                diagnosis.add_eye_examination_os(eye_examination_os=eye_examination)
                eye_image = EyeImage(
                    path_retina_contours_image=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'ImagesWithContours', f'Opht_cont_RET{row_os['ID']:03d}OS.jpg'),            
                )
                retina_image = Retina(
                    path_retina_image=os.path.join('..', 'data', 'load_data', 'FundusImages', f'RET{row_os['ID']:03d}OS.jpg')
                )
                contours_layers = Contours(
                    path_retina_cup_exp1=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row_os['ID']:03d}OS_cup_exp1.txt'),
                    path_retina_cup_exp2=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row_os['ID']:03d}OS_cup_exp2.txt'),
                    path_retina_disc_exp1=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row_os['ID']:03d}OS_disc_exp1.txt'),
                    path_retina_disc_exp2=os.path.join('..', 'data', 'load_data', 'ExpertsSegmentations', 'Contours', f'RET{row_os['ID']:03d}OS_disc_exp2.txt')
                )
                eye_examination.add_eye_image(eye_image=eye_image)
                eye_image.add_retina_image(retina_image=retina_image)
                eye_image.add_contours_layer(contours_layer=contours_layers)

df = convert_xls_to_df(os.path.join('..', 'data', 'load_data', 'ClinicalData', 'diagnosis_data.xlsx'), 'od')
diagnosises = load_data(df)
df_os = convert_xls_to_df(os.path.join('..', 'data', 'load_data', 'ClinicalData', 'diagnosis_data.xlsx'), 'os')
add_diagnosises_os(diagnosises=diagnosises, df_entry_os=df_os)

   ID  Age Gender Diagnosis  dioptre_1  dioptre_2  astigmatism  \
0   2   47      0         2       0.75      -1.75         90.0   
1   4   58      1         1        1.5      -1.75         85.0   
2   5   89      1         1      -0.75      -1.25        101.0   
3   6   69      0         2        1.0       -1.5         95.0   
4   7   22      1         2      -0.25        0.0          0.0   
5   8   67      1         2       <NA>      -0.75         20.0   
6   9   79      0         2       0.75       -1.5         95.0   
7  10   72      1         1       2.25       -1.5        105.0   
8  13   70      1         1        3.0       -1.0         65.0   
9  14   60      1         1       0.25       -0.5        155.0   

  Phakic/Pseudophakic  Pneumatic  Perkins  Pachymetry  Axial_Length  VF_MD  
0                   0       21.0     <NA>       586.0     23.639999  -0.07  
1                   0       <NA>     19.0       501.0     23.059999  -3.26  
2                   1       13.0     14.0 

VISUALIZAR UN DIAGNOSIS

In [4]:
print(diagnosises[9])

Diagnosis: (diagnosis_id: 14, diagnosis: 1, patient: Patient: (age=60, gender=1), eye_examination_od: EyeExamination: (dioptre_1: 0.25, dioptre_2: -0.5, astigmatism: 155.0, phakic_pseudophakic: 0, pneumatic: 11.0, perkins: 14.0, pachymetry: 505.0, axial_length: 23.809999465942383, vf_md: -11.369999885559082, eye_image: EyeImage: (path_retina_contourns_image='../data/load_data/ExpertsSegmentations/ImagesWithContours/Opht_cont_RET014OD.jpg', retina_image=Retina: (path_retina_image='../data/load_data/FundusImages/RET014OD.jpg'), contours_layer=Contours: (path_retina_cup_exp1='../data/load_data/ExpertsSegmentations/Contours/RET014OD_cup_exp1.txt', path_retina_cup_exp2='../data/load_data/ExpertsSegmentations/Contours/RET014OD_cup_exp2.txt', path_retina_disc_exp1='../data/load_data/ExpertsSegmentations/Contours/RET014OD_disc_exp1.txt', path_retina_disc_exp2='../data/load_data/ExpertsSegmentations/Contours/RET014OD_disc_exp2.txt'))), eye_examination_os: EyeExamination: (dioptre_1: 0.25, diopt